# 2. Exercise
역할 : 일본의 도시 안전 계획자이며 일본의 어느 지역에서 추가 지진 보강이 필요한지 분석하고 있습니다. 인구 밀도가 높고 지진에 취약한 지역은 어디인가요?

In [14]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

## 2-1. 대륙간 경계 데이터 불러오기

In [15]:
plate_boundaries = gpd.read_file("../../01_data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


In [16]:
# 지진 발생 데이터를 불러오기
earthquakes = pd.read_csv("../../01_data/earthquakes1970-2014.csv", parse_dates=['DateTime'])
earthquakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


In [47]:
# 배경맵에 대륙 경계 데이터 불러오기
m_7 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_7)

# 배경맵에 히트맵 추가하기
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_7)

m_7

## 2-2. 대륙간 경계판이 일본의 지진 발생 깊이와 연관이 있는지?

지진이 발생한 지점의 깊이와, 대륙간의 경계판이 서로 연관이 있는지 확인

In [48]:
# 대륙경계로 배경맵 만들기
m_8 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_8)
    
# 지진 강도로 구분해서 표현
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

for i in range(0, len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_8)
m_8

## 2-3. 인구밀도가 높은 일본 현을 확인

In [22]:
# 일본 현 경계 데이터를 불러오기
prefectures = gpd.read_file("../../01_data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.6533, 137.09546 3..."
Akita,"MULTIPOLYGON (((139.55725 39.2033, 139.55765 3..."
Aomori,"MULTIPOLYGON (((141.3986 40.92472, 141.39806 4..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


In [33]:
# 인구 데이터 불러오기
population = pd.read_csv("../../01_data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# 각 현에 거주하고있는 인구 대비 면적 계산하기
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# 밀도 표현하기
stats['density'] = stats['population'] / stats['area_sqkm']
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


In [49]:
# 배경맵 만들기
m_9 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)

# 코로플레스 지도를 통해서 인구밀도 시각화하기
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
          data=stats['density'],
          key_on='feature.id',
          fill_color='YlGnBu',
          legend_name='인구 밀도(per square kilometer)').add_to(m_9)

m_9

## 2-4. 인구 밀집도가 높은 현 중에서 지진이 자주 발생한 지점 확인

In [1]:
m_10 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# 지진 강도에 따라 색상 표현 변화
def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

# 코로플레스 지도 표현(일본 인구밀도를 현 기준으로 표현)
Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on='feature.id',
    fill_color='BuPu',
    legend_name='인구 밀도(per square kilometer)').add_to(m_10)

# 대륙 경계판도 추가하기
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_10)
    
# 버블맵으로 표현하기
for i in range(0, len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])
    ).add_to(m_10)
    
m_10

!['japan_earthquakes'](jp_earthquakes.png)

## 2-5. 결론
### 1) 도쿄 :
일본 수도인 도쿄는 일본 현 중에서 가장 높은 인구밀도를 갖고 있으며, 여러 지진이 발생하기도 했습니다.\
또한 대륙 경계판과 근접하므로 추가 지진 발생 위험이 잇따르고 있습니다.
### 2) 오사카 :
오사카는 도쿄에 비해 그리 높지 않은 인구밀도를 보여주고 있지만, 도쿄보다 상대적으로 진도가 높은 지진을 경험하였습니다.\
### 3) 가나가와 :
가나가와의 긴 해안(고밀도와 강진의 역사적 근접성 외에도)은 추가적인 쓰나미 위험에 대해 걱정하게 만들 수 있습니다.
